<a href="https://colab.research.google.com/github/maskot1977/tmd2022/blob/o3VQO0cT5iFiBFJV/tmd2022_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

「AI創薬・ケモインフォマティクス入門」講義資料　（講師：小寺正明）

2月11日(土)19:40～21:10 第3回「分子構造情報処理」

# 第3回　分子構造情報処理

# RDKitのインストール

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c rdkit rdkit python=3.7
import sys

sys.path.append("/usr/local/lib/python3.7/site-packages/")

In [ ]:
import rdkit

# 化合物データのダウンロード


In [ ]:
import urllib.request

# 化合物データのダウンロード
url = "https://raw.githubusercontent.com/maskot1977/tmd2020/main/data/data_1.csv"
urllib.request.urlretrieve(url, url.split("/")[-1])

# pandasのインポートとデータ読み込み

In [ ]:
import pandas as pd

# csvからのデータ読み込み
df = pd.read_csv("data_1.csv")
df

# RDKitを使った化学フォーマット変換

In [ ]:
# １つめのSMILES
smile = df["Open Babel SMILES"][0]
smile

In [ ]:
from rdkit import Chem

# SMILES を RDKit Mol に変換
mol = Chem.MolFromSmiles(smile)
mol

In [ ]:
# 名前をつける
mol.SetProp("_Name", "Satan Miracle Special Ultra Super Megaton Punch")

In [ ]:
# InChI に変換
inchi = Chem.MolToInchi(mol)
inchi

In [ ]:
# MolBlock形式に変換
molblock = Chem.MolToMolBlock(mol)
print(molblock)

In [ ]:
# 水素原子を明記する
mol = Chem.AddHs(mol)
molblock = Chem.MolToMolBlock(mol)
print(molblock)

In [ ]:
# 水素原子を省略する
mol = Chem.RemoveHs(mol)
molblock = Chem.MolToMolBlock(mol)
print(molblock)

In [ ]:
# ファイルに出力する
filename = "SatanMiracleSpecialUltraSuperMegatonPunch.mol"
print(Chem.MolToMolBlock(mol), file=open(filename, "w+"))

# 描画

## １分子の描画

In [ ]:
from rdkit.Chem import Draw

smile = df["Open Babel SMILES"][0]
mol = Chem.MolFromSmiles(smile)
mol.SetProp("_Name", smile)

# ノートブック上に描画
Draw.MolToImage(mol, legend=mol.GetProp("_Name"))

In [ ]:
# ファイルに出力する
filename = "SatanMiracleSpecialUltraSuperMegatonPunch.png"
Chem.Draw.MolToFile(mol, filename, legend=mol.GetProp("_Name"))

## 構造式描画をもっと詳しく

In [ ]:
# どの原子にどの番号がついているか表示したい
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D

view = rdMolDraw2D.MolDraw2DSVG(300, 350)
view.SetFontSize(0.9 * view.FontSize())
tm = rdMolDraw2D.PrepareMolForDrawing(mol)

option = view.drawOptions()
for atomidx, atom in enumerate(mol.GetAtoms()):
    symbol = mol.GetAtomWithIdx(atomidx).GetSymbol()
    option.atomLabels[atomidx] = symbol + str(atomidx)

view.DrawMolecule(tm, legend=mol.GetProp("_Name"))
view.FinishDrawing()
# 描画
svg = view.GetDrawingText()
SVG(svg)

In [ ]:
# 指定した原子をハイライトしたい
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D

colors = {6: (1, 0, 0), 8: (0, 1, 0), 10: (0, 0, 1)}
radii = {6: 0.25, 8: 0.50, 10: 0.2}

view = rdMolDraw2D.MolDraw2DSVG(300, 350)
view.SetFontSize(0.9 * view.FontSize())
tm = rdMolDraw2D.PrepareMolForDrawing(mol)
option = view.drawOptions()
option.multipleBondOffset = 0.07
option.padding = 0.11
option.legendFontSize = 20
view.DrawMolecule(
    tm,
    highlightAtoms=colors.keys(),
    highlightAtomColors=colors,
    highlightAtomRadii=radii,
    legend=mol.GetProp("_Name"),
)
view.FinishDrawing()
# 描画
svg = view.GetDrawingText()
SVG(svg)

## 複数分子の描画


In [ ]:
mols = []  # 複数の RDKit Mol を格納するリスト
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)  # SMILES を RDKit Mol に変換
    mol.SetProp("_Name", str(id))  # 名前をつける
    mols.append(mol)  # リストに格納する

In [ ]:
x_start = 0  # 表示したい最初の化合物
x_end = 100  # 表示したい最後の化合物の次
img = Draw.MolsToGridImage(
    mols[x_start:x_end],  # スライスで範囲を指定
    molsPerRow=10,  # 列の数
    subImgSize=(100, 100),  # 画像サイズ
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]],  # 注釈
)
display(img)

In [ ]:
# ファイルに出力
img.save("SatanMiracleSpecialUltraSuperMegatonPunch_grid.png")

# カノニカライズ（記述順序の標準化）

In [ ]:
cnt = 0
for smile in df["Open Babel SMILES"]:
    mol = Chem.MolFromSmiles(smile)  # SMILES を RDKit Mol Object に変換し
    new_smile = Chem.MolToSmiles(mol)  # それを SMILES に戻す
    if smile != new_smile:  # もとの SMILES と新しい SMILES が違っていたら表示する
        cnt += 1
        print(cnt, "\t", smile, "\t", new_smile)

# 電荷の中和

In [ ]:
from rdkit.Chem import MolStandardize

uc = MolStandardize.charge.Uncharger()

smile = "[NH3+][C@@H](Cc1c[nH]cn1)C(=O)[O-]"
before = Chem.MolFromSmiles(smile)
after = uc.uncharge(before)
smile2 = Chem.MolToSmiles(after)

print(smile, "=>", smile2)
Draw.MolsToGridImage([before, after])

# 脱塩

In [ ]:
from rdkit.Chem.SaltRemover import SaltRemover

remover = SaltRemover(defnData="[Cl,Br,Na]")

smile = "[NH3+][C@@H](Cc1c[nH]cn1)C(=O)[O-].[Na+].[Cl-]"
before = Chem.MolFromSmiles(smile)
after = remover.StripMol(before)
smile2 = Chem.MolToSmiles(after)

print(smile, "=>", smile2)
Draw.MolsToGridImage([before, after])

# 一番大きい分子だけ残す

In [ ]:
from rdkit.Chem import MolStandardize

lfc = MolStandardize.fragment.LargestFragmentChooser()

smile = "[NH3+][C@@H](Cc1c[nH]cn1)C(=O)[O-].[Na+].[Cl-].[O-]C(=O)CCC(=O)[O-]"
before = Chem.MolFromSmiles(smile)
after = lfc.choose(before)
smile2 = Chem.MolToSmiles(after)

print(smile, "=>", smile2)
Draw.MolsToGridImage([before, after])

# 前処理を全部やる

In [ ]:
from rdkit.Chem import MolStandardize
from rdkit.Chem.SaltRemover import SaltRemover

uc = MolStandardize.charge.Uncharger()
remover = SaltRemover(defnData="[Cl,Br,Na]")
lfc = MolStandardize.fragment.LargestFragmentChooser()

preprocessed_smiles = []
for smile in df["Open Babel SMILES"]:
    mol = Chem.MolFromSmiles(smile)
    smile = Chem.MolToSmiles(mol)
    mol = uc.uncharge(mol)
    mol = remover.StripMol(mol)
    mol = lfc.choose(mol)
    new_smile = Chem.MolToSmiles(mol)
    preprocessed_smiles.append(new_smile)
    if smile != new_smile:
        print(smile, new_smile)

# 部分構造検索

In [ ]:
from rdkit.Chem import Draw

smile = df["Open Babel SMILES"][0]
mol = Chem.MolFromSmiles(smile)
mol.SetProp("_Name", "Recoom Ultra Fighting Miracle Bomber")

# ノートブック上に描画
Draw.MolToImage(mol, legend=smile)

In [ ]:
# 検索したい部分構造パターン
smart = "CNC"
patt = Chem.MolFromSmarts(smart)
Draw.MolToImage(patt, legend=smart)

In [ ]:
# 指定した分子中で見つかったパターンのうち最初のもの
mol.GetSubstructMatch(patt)

In [ ]:
# 指定した原子をハイライトしたい
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D

view = rdMolDraw2D.MolDraw2DSVG(300, 350)
view.SetFontSize(0.9 * view.FontSize())
tm = rdMolDraw2D.PrepareMolForDrawing(mol)
option = view.drawOptions()
option.multipleBondOffset = 0.07
option.padding = 0.11
option.legendFontSize = 20
view.DrawMolecule(
    tm,
    highlightAtoms=mol.GetSubstructMatch(patt),
    # highlightAtomColors=colors,
    # highlightAtomRadii=radii,
    legend=str(mol.GetSubstructMatch(patt)),
)
view.FinishDrawing()
# 描画
svg = view.GetDrawingText()
SVG(svg)

In [ ]:
# 指定した分子中で見つかったパターン全部
mol.GetSubstructMatches(patt)

In [ ]:
# 指定した原子をハイライトしたい
matches = mol.GetSubstructMatches(patt)
tm = rdMolDraw2D.PrepareMolForDrawing(mol)
yoko = 640
tate = 280
n_col = 4
n_row = 2
view = rdMolDraw2D.MolDraw2DSVG(
    yoko, tate, int(yoko / n_col * 0.9), int(tate / n_row * 0.9)
)

view.DrawMolecules(
    [tm] * len(matches),
    highlightAtoms=matches,
    legends=[",".join([str(n) for n in match]) for match in matches],
)
# 描画
view.FinishDrawing()
svg = view.GetDrawingText()
SVG(svg)

In [ ]:
# 検索したい部分構造パターン
smart = "c1ccc2[nH]ccc2c1"
patt = Chem.MolFromSmarts(smart)
Draw.MolToImage(patt, legend=smart)

In [ ]:
mols = []
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt):  # パターンを含む分子のみ mols に格納する
        mols.append(mol)

len(mols)  # 格納された分子の数

In [ ]:
# 描画
matches = [mol.GetSubstructMatches(patt)[0] for mol in mols]

Draw.MolsToGridImage(
    mols,
    molsPerRow=5,
    subImgSize=(160, 160),
    highlightAtomLists=matches,
    legends=[x.GetProp("_Name") for x in mols],
)

In [ ]:
# 指定した部分構造の向きを揃えて描画したい
from rdkit.Chem import AllChem

mols = []
AllChem.Compute2DCoords(patt)  # 部分構造の2D座標を計算する
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt):  # パターンを含む分子のみ mols に格納する
        AllChem.GenerateDepictionMatching2DStructure(mol, patt)  # 向きを揃える
        mols.append(mol)

matches = [mol.GetSubstructMatches(patt)[0] for mol in mols]

len(mols)  # 格納された分子の数

# 描画
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(160, 160),
    highlightAtomLists=matches,
    legends=[x.GetProp("_Name") for x in mols],
)

## 部分構造検索の注意点


In [ ]:
# 検索したい部分構造パターン
smart = "c1ccccc1"
patt = Chem.MolFromSmarts(smart)
Draw.MolToImage(patt, legend=smart)

In [ ]:
ids1 = []
patt = Chem.MolFromSmarts("c1ccccc1")  # SMARTSを使ったパターン
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt):
        ids1.append(id)

print(len(ids1))

In [ ]:
ids2 = []
patt = Chem.MolFromSmiles("c1ccccc1")  # SMILESを使ったパターン
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt):
        ids2.append(id)

print(len(ids2))

In [ ]:
set(ids1) - set(ids2)  # SMARTSとSMILESで結果が異なることがある

In [ ]:
# 結果が異なる例を表示
for id in set(ids1) - set(ids2):
    smile = list(df[df["PCCDB-ID"] == id]["Open Babel SMILES"])[0]
    print(smile)
    mol = Chem.MolFromSmiles(smile)
    display(Draw.MolToImage(mol))

## 部分構造検索における立体化学



In [ ]:
mols = []
patt = Chem.MolFromSmiles("C[C@H](N)CO")
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt, useChirality=True):  # useChirality=True
        mols.append(mol)

print(len(mols))

x_start = 0
x_end = 5
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]],
)

In [ ]:
mols = []
patt = Chem.MolFromSmiles("C[C@@H](N)CO")
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    if mol.HasSubstructMatch(patt, useChirality=True):  # useChirality=True
        mols.append(mol)

print(len(mols))

x_start = 0
x_end = 5
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]],
)

# ３D構造の生成

In [ ]:
from rdkit.Chem import Draw

smile = df["Open Babel SMILES"][0]
mol = Chem.MolFromSmiles(smile)
mol.SetProp("_Name", smile)

# ノートブック上に描画
Draw.MolToImage(mol, legend=mol.GetProp("_Name"))

In [ ]:
print(Chem.MolToMolBlock(mol))

In [ ]:
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
mol = Chem.RemoveHs(mol)
print(Chem.MolToMolBlock(mol))

# 3Dの描画

参考記事：https://future-chem.com/py3dmol/

In [ ]:
!pip install py3Dmol

In [ ]:
import py3Dmol
from rdkit.Chem.Draw import IPythonConsole

view = py3Dmol.view(width=600, height=400, linked=False, viewergrid=(1, 1))
view.addModel(Chem.MolToMolBlock(mol), "sdf", {"keepH": False})
view.setStyle({"stick": {}}, viewer=(0, 0))
# view.setStyle({'sphere': {}}, viewer=(0, 0))
# view.setStyle({'line': {'linewidth': 50}}, viewer=(0,0))
view.setBackgroundColor("#000000", viewer=(0, 0))
view.show()

In [ ]:
mols = []  # 複数の RDKit Mol を格納するリスト
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)  # SMILES　を　RDKit Mol　に変換
    mol.SetProp("_Name", str(id))  # 名前をつける
    mols.append(mol)  # リストに格納する

add_mols = []
for m in mols[:6]:
    m_h = Chem.AddHs(m)
    AllChem.EmbedMolecule(m_h, AllChem.ETKDGv2())
    add_mols.append(m_h)
color_scheme = ["cyanCarbon", "blackCarbon", "ssPyMOL", "ssJmol", "Jmol", "default"]
view = py3Dmol.view(width=680, height=400, viewergrid=(2, 3), linked=False)
for (n, (i, j)), c in zip(
    enumerate([(a, b) for a in range(2) for b in range(3)]), color_scheme
):
    mb = Chem.MolToMolBlock(add_mols[n])
    view.addModel(mb, "sdf", {"keepH": False}, viewer=(i, j))
    view.setStyle({"stick": {"colorscheme": c}}, viewer=(i, j))
view.zoomTo()
view.show()

# Recap

Recapは、実験室でよく使われる反応を模倣した化学変換のセットを使って、分子を一連の合理的なフラグメントに分解します。

Lewell, X.Q.; Judd, D.B.; Watson, S.P.; Hann, M.M. “RECAP-Retrosynthetic Combinatorial Analysis Procedure: A Powerful New Technique for Identifying Privileged Molecular Fragments with Useful Applications in Combinatorial Chemistry” J. Chem. Inf. Comp. Sci. 38:511-22 (1998).

In [ ]:
smile = df["Open Babel SMILES"][0]
print(smile)
mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(mol)

In [ ]:
from rdkit.Chem import Recap

# Recap アルゴリズムによる化学フラグメント分解
hierarch = Recap.RecapDecompose(mol)
ks = hierarch.children
ks

In [ ]:
# 得られたフラグメントの描画
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(k) for k in ks.keys()],
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=list(ks.keys()),
)

# BRICS

BRICSは合成的に利用可能な結合に沿って分子をフラグメント化するもう一つの方法です

Degen, J.; Wegscheid-Gerlach, C.; Zaliani, A; Rarey, M. "On the Art of Compiling and Using ‘Drug-Like’ Chemical Fragment Spaces." ChemMedChem 3:1503–7 (2008)

In [ ]:
from rdkit.Chem import BRICS

# BRICS アルゴリズムによる化学フラグメント分解
fragments = BRICS.BRICSDecompose(mol)

# 得られたフラグメントの描画
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(k) for k in list(fragments)],
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=list(fragments),
)

In [ ]:
# フラグメント化したい分子の集合を作る
mols = []
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)
    mol.SetProp("_Name", str(id))
    mols.append(mol)

x_start = 0
x_end = 10
Draw.MolsToGridImage(
    mols[x_start:x_end],
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=[x.GetProp("_Name") for x in mols[x_start:x_end]],
)

In [ ]:
# 分子の集合から、フラグメントの集合を得る
allfrags = set()
for mol in mols[x_start:x_end]:
    fragment = BRICS.BRICSDecompose(mol)
    allfrags.update(fragment)

print(len(allfrags))  # 得られたフラグメントの数

# フラグメントの集合の描画
Draw.MolsToGridImage(
    [Chem.MolFromSmiles(f) for f in allfrags],
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=list(allfrags),
)

In [ ]:
# フラグメントの集合にBRICSのルールを適用して新しい分子を生成する
fragms = [Chem.MolFromSmiles(x) for x in allfrags]

products = []
for mol in BRICS.BRICSBuild(fragms):
    mol.UpdatePropertyCache(strict=False)
    products.append(mol)
    if len(products) >= 25:
        break

# 得られた新しい分子の描画
Draw.MolsToGridImage(
    products,
    molsPerRow=5,
    subImgSize=(200, 200),
    legends=[Chem.MolToSmiles(mol) for mol in products],
)

# 分子ごとの化学記述子

In [ ]:
smile = df["Open Babel SMILES"][0]
print(smile)
mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(mol)

トポロジカル極性表面積

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.TPSA(mol)

Crippenらによる原子ベースのLogP指標

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.MolLogP(mol)

分子量

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.MolWt(mol)

非水素原子の数

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.HeavyAtomCount(mol)

水素結合アクセプターの数

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.NumHAcceptors(mol)

水素結合ドナーの数

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.NumHDonors(mol)

へテロ元素の数

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.NumHeteroatoms(mol)

回転可能な結合数

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.NumRotatableBonds(mol)

環の数

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.RingCount(mol)

全炭素数におけるsp3炭素の割合

In [ ]:
from rdkit.Chem import Descriptors

Descriptors.FractionCSP3(mol)

## 計算可能な記述子を列挙してマトリクスを作る


In [ ]:
desc_names = [x[0] for x in Descriptors._descList if x[0]]
print(desc_names)

In [ ]:
len(desc_names)

In [ ]:
from rdkit.Chem import MolStandardize
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors

uc = MolStandardize.charge.Uncharger()
remover = SaltRemover(defnData="[Cl,Br,Na]")
lfc = MolStandardize.fragment.LargestFragmentChooser()
calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

matrix = []
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    row = []
    row.append(id)
    mol = Chem.MolFromSmiles(smile)
    mol = uc.uncharge(mol)
    mol = remover.StripMol(mol)
    mol = lfc.choose(mol)
    row.append(Chem.MolToSmiles(mol))
    for d in calc.CalcDescriptors(mol):
        row.append(d)
    matrix.append(row)

desc_df = pd.DataFrame(matrix, columns=["ID", "SMILES"] + desc_names)
desc_df

# 原子ごとの物理化学的特徴


In [ ]:
smile = df["Open Babel SMILES"][0]
print(smile)
mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(mol)

In [ ]:
from rdkit.Chem import AllChem

AllChem.ComputeGasteigerCharges(mol)
for id, atom in enumerate(mol.GetAtoms()):
    print(id + 1, atom.GetSymbol(), atom.GetProp("_GasteigerCharge"))

In [ ]:
import os

from rdkit import RDConfig
from rdkit.Chem import ChemicalFeatures

fdefName = os.path.join(RDConfig.RDDataDir, "BaseFeatures.fdef")
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

for i, feat in enumerate(factory.GetFeaturesForMol(mol)):
    print(i, feat.GetFamily(), feat.GetType(), feat.GetAtomIds())

## SASA (solvent accessible surface area)



In [ ]:
from rdkit.Chem import rdFreeSASA

mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)

radii = rdFreeSASA.classifyAtoms(mol)
rdFreeSASA.CalcSASA(mol, radii)

In [ ]:
atoms = mol.GetAtoms()
for i in range(len(atoms)):
    print(i, atoms[i].GetSymbol(), atoms[i].GetProp("SASA"))

# 2Dファーマコフォアフィンガープリント

In [ ]:
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory

fdefName = os.path.join(RDConfig.RDDataDir, "BaseFeatures.fdef")
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory, minPointCount=2, maxPointCount=3)

# 距離をビン分割して、それぞれのファーマコフォアに一意の整数値のIDを割り当てる
sigFactory.SetBins([(0, 2), (2, 5), (5, 8)])
sigFactory.Init()
sigFactory.GetSigSize()

In [ ]:
# 2Dファーマコフォアフィンガープリントの作成
from rdkit.Chem.Pharm2D import Generate

fp = Generate.Gen2DFingerprint(mol, sigFactory)
len(fp)  # フィンガープリントの長さ

In [ ]:
fp.GetNumOnBits()  # 立っているビットの数

In [ ]:
# 立っているビットの内訳
for bit in list(fp.GetOnBits()):
    print(bit, sigFactory.GetBitDescription(bit))

Gobbi & Poppinger 定義の２Dファーマコフォアを用いる

Gobbi, A. & Poppinger, D. "Genetic optimization of combinatorial libraries." Biotechnology and Bioengineering 61:47-54 (1998).

In [ ]:
from rdkit import Chem
from rdkit.Chem.Pharm2D import Generate, Gobbi_Pharm2D

fp = Generate.Gen2DFingerprint(mol, Gobbi_Pharm2D.factory)
len(fp)  # フィンガープリントの長さ

In [ ]:
fp.GetNumOnBits()  # 立っているビットの数

In [ ]:
# 立っているビットの内訳
for bit in list(fp.GetOnBits()):
    print(bit, Gobbi_Pharm2D.factory.GetBitDescription(bit))

# フィンガープリントを用いた類似分子検索



In [ ]:
smile = df["Open Babel SMILES"][0]
print(smile)
query_mol = Chem.MolFromSmiles(smile)
Draw.MolToImage(query_mol)

In [ ]:
from rdkit import DataStructs
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem, MACCSkeys
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem.Fingerprints import FingerprintMols

# フィンガープリントいろいろ
get_fp = lambda mol: MACCSkeys.GenMACCSKeys(mol)
# get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
# get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useFeatures=True)
# get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 4, nBits=1024)
# get_fp = lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 4, nBits=1024, useFeatures=True)
# get_fp = lambda mol: Pairs.GetAtomPairFingerprintAsBitVect(mol)
# get_fp = lambda mol: FingerprintMols.FingerprintMol(mol) #Topological Fingerprint
# get_fp = lambda mol: Torsions.GetTopologicalTorsionFingerprintAsIntVect(mol)
# get_fp = lambda mol: pyAvalonTools.GetAvalonFP(mol)

# 類似性指標もいろいろ
metric = DataStructs.TanimotoSimilarity
# metric = DataStructs.DiceSimilarity
# metric = DataStructs.CosineSimilarity
# metric = DataStructs.SokalSimilarity
# metric = DataStructs.RusselSimilarity
# metric = DataStructs.KulczynskiSimilarity
# metric = DataStructs.McConnaugheySimilarity

In [ ]:
query_fp = get_fp(query_mol)

mols = []
fps = []  # 複数のリンガープリントを格納するリスト
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)  # SMILES　を　RDKit Mol　に変換
    mol.SetProp("_Name", str(id))  # 名前をつける
    fp = get_fp(mol)
    fps.append(fp)  # リストに格納する
    mols.append(mol)

results = []
for i, fp in enumerate(fps):
    results.append([DataStructs.FingerprintSimilarity(query_fp, fp, metric=metric), i])

searched_mols = []
similarity_values = []
for sim, i in reversed(sorted(results)):
    searched_mols.append(mols[i])
    similarity_values.append(sim)

x_start = 0  # 表示したい最初の化合物
x_end = 10  # 表示したい最後の化合物の次
Draw.MolsToGridImage(
    searched_mols[x_start:x_end],  # スライスで範囲を指定
    molsPerRow=5,  # 列の数
    subImgSize=(200, 200),  # 画像サイズ
    legends=[str(v) for v in similarity_values],  # 注釈
)

# 最大共通部分構造

In [ ]:
from rdkit.Chem import rdFMCS

mcs = rdFMCS.FindMCS([query_mol, mol], timeout=60)
mcs.numAtoms, mcs.numBonds  # 最大共通部分構造に含まれる原子数、結合数

In [ ]:
mol_mcs = Chem.MolFromSmarts(mcs.smartsString)
mcs1 = query_mol.GetSubstructMatch(mol_mcs)
mcs2 = mol.GetSubstructMatch(mol_mcs)
mcs1, mcs2  # それぞれで一致した原子番号の一覧

In [ ]:
Draw.MolsToGridImage(
    [query_mol, mol],
    molsPerRow=2,
    subImgSize=(400, 400),
    highlightAtomLists=[mcs1, mcs2],
)

In [ ]:
# 最大共通部分構造の大きさに基づいた類似性検索

mols = []
for smile, id in zip(df["Open Babel SMILES"], df["PCCDB-ID"]):
    mol = Chem.MolFromSmiles(smile)  # SMILES　を　RDKit Mol　に変換
    mol.SetProp("_Name", str(id))  # 名前をつける
    mols.append(mol)

results = []
for i, mol in enumerate(mols):
    mcs = rdFMCS.FindMCS([query_mol, mol], timeout=60)
    results.append([mcs.numAtoms, mcs.numBonds, i])

searched_mols = []
similarity_values = []
highlight_atom_lists = []
for n_atom, n_bond, i in reversed(sorted(results)):
    searched_mols.append(mols[i])
    similarity_values.append(str(n_atom) + "," + str(n_bond))
    mcs = rdFMCS.FindMCS([query_mol, mols[i]], timeout=60)
    mol_mcs = Chem.MolFromSmarts(mcs.smartsString)
    mcs2 = mols[i].GetSubstructMatch(mol_mcs)
    highlight_atom_lists.append(mcs2)

x_start = 0  # 表示したい最初の化合物
x_end = 10  # 表示したい最後の化合物の次
Draw.MolsToGridImage(
    searched_mols[x_start:x_end],  # スライスで範囲を指定
    molsPerRow=5,  # 列の数
    subImgSize=(160, 160),  # 画像サイズ
    highlightAtomLists=highlight_atom_lists[x_start:x_end],
    legends=similarity_values,  # 注釈
)